In [14]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import lightgbm as lgb
import pandas as pd
import sys 
sys.path.append("E:\\NoteBook\\jhlib")

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 100)

**自定义函数**

In [3]:
# 官网的评价标准
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0
    

# 添加特征列函数
def get_dict(df, feat_col, agg_cal):
    dic = df.groupby("fragment_id")[feat_col].agg(agg_cal).to_dict()
    feat_name = '_'.join([feat_col, agg_cal])
    return feat_name, dic

# 自定义损失函数
def lgb_acc_combo(y_pred, label):
    y_true = label.get_label()
    sample_num = len(y_true) 
    class_num  = len(y_pred) // sample_num
    y_pred = y_pred.reshape(class_num, sample_num) 
    pred   = y_pred.argmax(axis=0)
    score_ = []
    for i in range(sample_num): 
        score_.append(acc_combo(y_true[i],pred[i]))
    return 'lgb_acc_combo',np.mean(score_),True

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
data_path = "E:\\DataSets\\keisci\\sensor\\"

In [5]:
data = pd.read_csv(data_path+"sensor_train\\sensor_train.csv")
data.head(2)

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0


In [7]:
data_y = data[["fragment_id","behavior_id"]].drop_duplicates().set_index(["fragment_id"])["behavior_id"]
data_x = data.drop("behavior_id", axis=1)

In [8]:
from tsfresh import extract_relevant_features
data_feat = extract_relevant_features(data_x, y=data_y, column_id='fragment_id', column_sort='time_point')
data_feat.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 30/30 [07:20<00:00, 14.68s/it]


variable,acc_yg__minimum,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""min""",acc_yg__quantile__q_0.1,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""",acc_yg__quantile__q_0.2,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",acc_yg__sum_of_reoccurring_data_points,acc_yg__quantile__q_0.3,"acc_yg__cwt_coefficients__coeff_7__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_5__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_8__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_4__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_3__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_2__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_9__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_1__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_11__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_13__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_14__w_20__widths_(2, 5, 10, 20)","acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""",acc_yg__mean,acc_yg__quantile__q_0.4,acc_yg__median,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",acc_yg__c3__lag_3,"acc_yg__linear_trend__attr_""intercept""",acc_yg__c3__lag_2,acc_yg__quantile__q_0.6,acc_yg__c3__lag_1,acc_yg__sum_values,"acc_yg__fft_coefficient__attr_""real""__coeff_0",acc_yg__quantile__q_0.7,"acc_yg__cwt_coefficients__coeff_10__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_11__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_12__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_7__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_13__w_10__widths_(2, 5, 10, 20)","acc_yg__fft_coefficient__attr_""abs""__coeff_0","acc_yg__cwt_coefficients__coeff_6__w_10__widths_(2, 5, 10, 20)",acc_yg__quantile__q_0.8,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","acc_yg__cwt_coefficients__coeff_5__w_5__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_14__w_10__widths_(2, 5, 10, 20)",...,acc_xg__large_standard_deviation__r_0.6000000000000001,acc_xg__large_standard_deviation__r_0.65,acc_xg__symmetry_looking__r_0.9500000000000001,acc_xg__symmetry_looking__r_0.9,acc_xg__symmetry_looking__r_0.8500000000000001,acc_xg__symmetry_looking__r_0.8,acc_xg__symmetry_looking__r_0.75,acc_xg__symmetry_looking__r_0.7000000000000001,acc_xg__symmetry_looking__r_0.65,acc_xg__symmetry_looking__r_0.6000000000000001,acc_xg__symmetry_looking__r_0.55,acc_xg__symmetry_looking__r_0.5,acc_xg__symmetry_looking__r_0.45,acc_xg__symmetry_looking__r_0.4,acc_xg__large_standard_deviation__r_0.9500000000000001,acc_xg__large_standard_deviation__r_0.9,acc_xg__large_standard_deviation__r_0.8500000000000001,acc_xg__large_standard_deviation__r_0.8,acc_z__large_standard_deviation__r_0.35000000000000003,acc_x__symmetry_looking__r_0.9500000000000001,acc_yg__large_standard_deviation__r_0.55,acc_z__large_standard_deviation__r_0.1,acc_z__large_standard_deviation__r_0.05,acc_x__symmetry_looking__r_0.65,acc_yg__symmetry_looking__r_0.9500000000000001,acc_yg__symmetry_looking__r_0.9,acc_yg__symmetry_looking__r_0.8500000000000001,acc_yg__symmetry_looking__r_0.8,acc_yg__symmetry_looking__r_0.75,acc_yg__symmetry_looking__r_0.7000000000000001,acc_yg__symmetry_looking__r_0.65,acc_yg__symmetry_looking__r_0.6000000000000001,acc_yg__symmetry_looking__r_0.55,acc_yg__symmetry_

In [10]:
data_feat.to_csv("E:\\DataSets\\keisci\\sensor\\middle_data\\data_feat.csv")

In [46]:
data_feat = pd.read_csv("E:\\DataSets\\keisci\\sensor\\middle_data\\data_feat.csv")

In [29]:
data["acc"] = (data['acc_x'] ** 2 + data['acc_y'] ** 2 + data['acc_z'] ** 2) ** 0.5
data['accg'] = (data['acc_xg'] ** 2 + data['acc_yg'] ** 2 + data['acc_zg'] ** 2) ** 0.5

data["acc_x_y"] = data["acc_x"] - data["acc_y"]
data["acc_x_z"] = data["acc_x"] - data["acc_z"]
data["acc_y_z"] = data["acc_y"] - data["acc_z"]
data["acc_xg_yg"] = data["acc_xg"] - data["acc_yg"]
data["acc_xg_zg"] = data["acc_xg"] - data["acc_zg"]
data["acc_yg_zg"] = data["acc_yg"] - data["acc_zg"]

data.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,acc,accg,acc_x_y,acc_x_z,acc_y_z,acc_xg_yg,acc_xg_zg,acc_yg_zg
_temporary_index_column,,,,,,,,,,,,,,,,,
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,0.435890,9.902020,0.6,0.2,-0.4,-3.9,-8.2,-4.3
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,0.412311,9.633795,0.1,0.5,0.4,-4.3,-8.0,-3.7
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0.316228,10.147413,0.1,-0.2,-0.3,-3.7,-8.1,-4.4
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0.519615,8.635392,0.2,0.6,0.4,-3.9,-6.4,-2.5
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0.640312,10.104949,-0.1,-0.5,-0.4,-3.8,-8.0,-4.2


In [30]:
data_feat2 = data.loc[:,["fragment_id","behavior_id"]].drop_duplicates()
feat_cols = [x for x in list(data.columns) if x not in ["fragment_id","behavior_id","time_point"]]
for feat_col in feat_cols:
    for agg_cal in ["max", "min","mean","skew","std","median","sum"]:
        feat_name, dic = get_dict(data, feat_col, agg_cal)
        data_feat2[feat_name] = data_feat2["fragment_id"].map(dic).values
#     data_feat2[feat_col+"_gap"] = data_feat2[feat_col+"_max"] - data_feat2[feat_col+"_min"]

In [56]:
data_feat1 = data_feat.rename(columns={"id":"fragment_id"})

In [57]:
data_feat1.head()

,fragment_id,acc_yg__minimum,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""min""",acc_yg__quantile__q_0.1,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""",acc_yg__quantile__q_0.2,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",acc_yg__sum_of_reoccurring_data_points,acc_yg__quantile__q_0.3,"acc_yg__cwt_coefficients__coeff_7__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_5__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_8__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_4__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_3__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_2__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_9__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_1__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_11__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_13__w_20__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_14__w_20__widths_(2, 5, 10, 20)","acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""",acc_yg__mean,acc_yg__quantile__q_0.4,acc_yg__median,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",acc_yg__c3__lag_3,"acc_yg__linear_trend__attr_""intercept""",acc_yg__c3__lag_2,acc_yg__quantile__q_0.6,acc_yg__c3__lag_1,acc_yg__sum_values,"acc_yg__fft_coefficient__attr_""real""__coeff_0",acc_yg__quantile__q_0.7,"acc_yg__cwt_coefficients__coeff_10__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_11__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_12__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_7__w_10__widths_(2, 5, 10, 20)","acc_yg__cwt_coefficients__coeff_13__w_10__widths_(2, 5, 10, 20)","acc_yg__fft_coefficient__attr_""abs""__coeff_0","acc_yg__cwt_coefficients__coeff_6__w_10__widths_(2, 5, 10, 20)",acc_yg__quantile__q_0.8,"acc_yg__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","acc_yg__cwt_coefficients__coeff_5__w_5__widths_(2, 5, 10, 20)",...,acc_xg__large_standard_deviation__r_0.6000000000000001,acc_xg__large_standard_deviation__r_0.65,acc_xg__symmetry_looking__r_0.9500000000000001,acc_xg__symmetry_looking__r_0.9,acc_xg__symmetry_looking__r_0.8500000000000001,acc_xg__symmetry_looking__r_0.8,acc_xg__symmetry_looking__r_0.75,acc_xg__symmetry_looking__r_0.7000000000000001,acc_xg__symmetry_looking__r_0.65,acc_xg__symmetry_looking__r_0.6000000000000001,acc_xg__symmetry_looking__r_0.55,acc_xg__symmetry_looking__r_0.5,acc_xg__symmetry_looking__r_0.45,acc_xg__symmetry_looking__r_0.4,acc_xg__large_standard_deviation__r_0.9500000000000001,acc_xg__large_standard_deviation__r_0.9,acc_xg__large_standard_deviation__r_0.8500000000000001,acc_xg__large_standard_deviation__r_0.8,acc_z__large_standard_deviation__r_0.35000000000000003,acc_x__symmetry_looking__r_0.9500000000000001,acc_yg__large_standard_deviation__r_0.55,acc_z__large_standard_deviation__r_0.1,acc_z__large_standard_deviation__r_0.05,acc_x__symmetry_looking__r_0.65,acc_yg__symmetry_looking__r_0.9500000000000001,acc_yg__symmetry_looking__r_0.9,acc_yg__symmetry_looking__r_0.8500000000000001,acc_yg__symmetry_looking__r_0.8,acc_yg__symmetry_looking__r_0.75,acc_yg__symmetry_looking__r_0.7000000000000001,acc_yg__symmetry_looking__r_0.65,acc_yg__symmetry_looking__r_0.6000000000000001,acc_yg__symmetry_looking__r_0.55,acc_yg__symmetry_looking__r_0.5,acc_yg__symmetry_looking__r_0.45,acc_x__symmetr

In [31]:
data_feat2.set_index("fragment_id",inplace=True)
data_feat2.head()

,behavior_id,acc_x_max,acc_x_min,acc_x_mean,acc_x_skew,acc_x_std,acc_x_median,acc_x_sum,acc_y_max,acc_y_min,acc_y_mean,acc_y_skew,acc_y_std,acc_y_median,acc_y_sum,acc_z_max,acc_z_min,acc_z_mean,acc_z_skew,acc_z_std,acc_z_median,acc_z_sum,acc_xg_max,acc_xg_min,acc_xg_mean,acc_xg_skew,acc_xg_std,acc_xg_median,acc_xg_sum,acc_yg_max,acc_yg_min,acc_yg_mean,acc_yg_skew,acc_yg_std,acc_yg_median,acc_yg_sum,acc_zg_max,acc_zg_min,acc_zg_mean,acc_zg_skew,acc_zg_std,acc_zg_median,acc_zg_sum,acc_max,acc_min,acc_mean,acc_skew,acc_std,acc_median,acc_sum,accg_max,accg_min,accg_mean,accg_skew,accg_std,accg_median,accg_sum,acc_x_y_max,acc_x_y_min,acc_x_y_mean,acc_x_y_skew,acc_x_y_std,acc_x_y_median,acc_x_y_sum,acc_x_z_max,acc_x_z_min,acc_x_z_mean,acc_x_z_skew,acc_x_z_std,acc_x_z_median,acc_x_z_sum,acc_y_z_max,acc_y_z_min,acc_y_z_mean,acc_y_z_skew,acc_y_z_std,acc_y_z_median,acc_y_z_sum,acc_xg_yg_max,acc_xg_yg_min,acc_xg_yg_mean,acc_xg_yg_skew,acc_xg_yg_std,acc_xg_yg_median,acc_xg_yg_sum,acc_xg_zg_max,acc_xg_zg_min,acc_xg_zg_mean,acc_xg_zg_skew,acc_xg_zg_std,acc_xg_zg_median,acc_xg_zg_sum,acc_yg_zg_max,acc_yg_zg_min,acc_yg_zg_mean,acc_yg_zg_skew,acc_yg_zg_std,acc_yg_zg_median,acc_yg_zg_sum
fragment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0.4,-0.9,-0.001754,-1.523203,0.194102,0.0,-0.1,0.2,-0.3,-0.001754,-0.497876,0.085547,-0.0,-0.1,0.7,-0.8,0.003509,-0.283199,0.294573,-0.0,0.2,1.3,-0.2,0.677193,-0.909003,0.200016,0.7,38.6,5.2,4.4,4.773684,0.284234,0.158707,4.8,272.1,9.3,7.2,8.508772,-0.746650,0.338738,8.5,485.0,0.953939,0.000000,0.281656,1.141795,0.225918,0.223607,16.054409,10.560776,8.635392,9.784581,-0.677138,0.293469,9.786726,557.721098,0.6,-0.8,1.947760e-18,-0.180893,0.226779,0.0,1.110223e-16,0.7,-0.8,-0.005263,0.071336,0.334034,0.0,-0.3,0.9,-0.9,-0.005263,0.082254,0.297862,0.0,-0.3,-3.6,-5.2,-4.096491,-1.104853,0.284708,-4.10,-233.5,-6.4,-9.5,-7.831579,-0.367830,0.436773,-7.8,-446.4,-2.5,-4.5,-3.735088,0.525371,0.388445,-3.8,-212.9
1,0,0.2,-0.5,-0.025000,-1.350853,0.109959,-0.0,-1.4,0.2,-0.1,0.026786,0.434311,0.058748,0.0,1.5,0.6,-0.5,0.078571,0.129004,0.184602,0.1,4.4,1.0,0.4,0.678571,-0.199871,0.126080,0.7,38.0,5.1,4.6,4.830357,0.411559,0.123465,4.8,270.5,9.4,8.1,8.519643,0.917398,0.241525,8.5,477.1,0.714143,0.000000,0.186749,1.626938,0.147525,0.141421,10.457923,10.521407,9.441398,9.819616,1.053017,0.202188,9.791067,549.898472,0.3,-0.6,-5.178571e-02,-0.894307,0.147699,0.0,-2.900000e+00,0.2,-0.6,-0.103571,-0.796679,0.186805,-0.1,-5.8,0.6,-0.5,-0.051786,0.272607,0.188767,0.0,-2.9,-3.8,-4.6,-4.151786,-0.268876,0.165135,-4.15,-232.5,-7.2,-8.9,-7.841071,-0.840256,0.283983,-7.8,-439.1,-3.0,-4.7,-3.689286,-0.431380,0.299502,-3.7,-206.6
2,0,0.7,-0.5,-0.028070,0.489693,0.239609,-0.0,-1.6,0.2,-0.6,-0.005263,-1.635879,0.136828,0.0,-0.3,1.2,-1.3,0.024561,-0.770588,0.396989,0.0,1.4,1.8,0.2,0.792982,0.853589,0.339569,0.7,45.2,5.4,4.4,4.952632,-0.081638,0.234601,4.9,282.3,9.8,7.2,8.394737,-0.103604,0.433186,8.4,478.5,1.513275,0.000000,0.363894,1.890966,0.316852,0.244949,20.741932,10.813418,8.823831,9.790638,-0.205059,0.355424,9.827004,558.066365,1.3,-0.6,-2.280702e-02,1.438085,0.313402,-0.1,-1.300000e+00,1.9,-1.6,-0.052632,0.469882,0.516204,-0.1,-3.0,1.5,-1.5,-0.029825,-0.005008,0.415746,0.0,-1.7,-2.9,-4.8,-4.159649,1.058967,0.413115,-4.20,-237.1,-5.9,-9.0,-7.601754,0.770910,0.605183,-7.8,-433.3,-2.0,-5.3,-3.442105,-0.109303,0.538831,-3.5,-196.2
3,0,0.5,-0.4,0.001818,0.072280,0.178980,0.0,0.1,0.4,-0.3,0.023636,1.126930,0.121661,-0.0,1.3,1.7,-0.8,0.094545,1.413450,0.496438,0.0,5.2,1.2,-0.6,0.410909,-0.555181,0.319532,0.4,22.6,5.5,4.5,5.049091,-0.054651,0.196141,5.0,277.7,9.5,7.5,8.340000,0.472631,0.414371,8.3,458.7,1.791647,0.000000,0.400589,1.973559,0.373524,0.282843,22.032386,10.585367,8.828363,9.767121,0.193839,0.360655,9.780082,537.191665,0.7,-0.8,-2.181818e-02,-0.394423,0.270590,0.0,-1.200000e+00,0.7,-2.1,-0.092727,-1.646655,0.566946,0.1,-5.1,1.1,-1.6,-0.070909,-1.041841,0.466537,0.0,-3

In [59]:
fd = pd.merge(data_feat2, data_feat1,how="inner", on="fragment_id")
fd.head()

,fragment_id,behavior_id,acc_x_max,acc_x_min,acc_x_mean,acc_x_skew,acc_x_std,acc_x_median,acc_x_sum,acc_y_max,acc_y_min,acc_y_mean,acc_y_skew,acc_y_std,acc_y_median,acc_y_sum,acc_z_max,acc_z_min,acc_z_mean,acc_z_skew,acc_z_std,acc_z_median,acc_z_sum,acc_xg_max,acc_xg_min,acc_xg_mean,acc_xg_skew,acc_xg_std,acc_xg_median,acc_xg_sum,acc_yg_max,acc_yg_min,acc_yg_mean,acc_yg_skew,acc_yg_std,acc_yg_median,acc_yg_sum,acc_zg_max,acc_zg_min,acc_zg_mean,acc_zg_skew,acc_zg_std,acc_zg_median,acc_zg_sum,acc_max,acc_min,acc_mean,acc_skew,acc_std,acc_median,...,acc_xg__large_standard_deviation__r_0.6000000000000001,acc_xg__large_standard_deviation__r_0.65,acc_xg__symmetry_looking__r_0.9500000000000001,acc_xg__symmetry_looking__r_0.9,acc_xg__symmetry_looking__r_0.8500000000000001,acc_xg__symmetry_looking__r_0.8,acc_xg__symmetry_looking__r_0.75,acc_xg__symmetry_looking__r_0.7000000000000001,acc_xg__symmetry_looking__r_0.65,acc_xg__symmetry_looking__r_0.6000000000000001,acc_xg__symmetry_looking__r_0.55,acc_xg__symmetry_looking__r_0.5,acc_xg__symmetry_looking__r_0.45,acc_xg__symmetry_looking__r_0.4,acc_xg__large_standard_deviation__r_0.9500000000000001,acc_xg__large_standard_deviation__r_0.9,acc_xg__large_standard_deviation__r_0.8500000000000001,acc_xg__large_standard_deviation__r_0.8,acc_z__large_standard_deviation__r_0.35000000000000003,acc_x__symmetry_looking__r_0.9500000000000001,acc_yg__large_standard_deviation__r_0.55,acc_z__large_standard_deviation__r_0.1,acc_z__large_standard_deviation__r_0.05,acc_x__symmetry_looking__r_0.65,acc_yg__symmetry_looking__r_0.9500000000000001,acc_yg__symmetry_looking__r_0.9,acc_yg__symmetry_looking__r_0.8500000000000001,acc_yg__symmetry_looking__r_0.8,acc_yg__symmetry_looking__r_0.75,acc_yg__symmetry_looking__r_0.7000000000000001,acc_yg__symmetry_looking__r_0.65,acc_yg__symmetry_looking__r_0.6000000000000001,acc_yg__symmetry_looking__r_0.55,acc_yg__symmetry_looking__r_0.5,acc_yg__symmetry_looking__r_0.45,acc_x__symmetry_looking__r_0.7000000000000001,acc_x__symmetry_looking__r_0.75,acc_x__symmetry_looking__r_0.8,acc_x__symmetry_looking__r_0.8500000000000001,acc_x__symmetry_looking__r_0.9,acc_yg__large_standard_deviation__r_0.9500000000000001,acc_yg__large_standard_deviation__r_0.9,acc_yg__large_standard_deviation__r_0.8500000000000001,acc_yg__large_standard_deviation__r_0.8,acc_yg__large_standard_deviation__r_0.75,acc_yg__large_standard_deviation__r_0.7000000000000001,acc_yg__large_standard_deviation__r_0.65,acc_yg__large_standard_deviation__r_0.6000000000000001,acc_yg__large_standard_deviation__r_0.5,acc_xg__large_standard_deviation__r_0.7000000000000001
0,0,0,0.4,-0.9,-0.001754,-1.523203,0.194102,0.0,-0.1,0.2,-0.3,-0.001754,-0.497876,0.085547,-0.0,-0.1,0.7,-0.8,0.003509,-0.283199,0.294573,-0.0,0.2,1.3,-0.2,0.677193,-0.909003,0.200016,0.7,38.6,5.2,4.4,4.773684,0.284234,0.158707,4.8,272.1,9.3,7.2,8.508772,-0.746650,0.338738,8.5,485.0,0.953939,0.000000,0.281656,1.141795,0.225918,0.223607,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0.2,-0.5,-0.025000,-1.350853,0.109959,-0.0,-1.4,0.2,-0.1,0.026786,0.434311,0.058748,0.0,1.5,0.6,-0.5,0.078571,0.129004,0.184602,0.1,4.4,1.0,0.4,0.678571,-0.199871,0.126080,0.7,38.0,5.1,4.6,4.830357,0.411559,0.123465,4.8,270.5,9.4,8.1,8.519643,0.917398,0.241525,8.5,477.1,0.714143,0.000000,0.186749,1.626938,0.147525,0.141421,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,0.7,-0.5,-0.028070,0.489693,0.239609,-0.0,-1.6,0.2,-0.6,-0.005263,-1.635879,0.136828,0.0,-0.3,1.2,-1.3,0.024561,-0.770588,0.396989,0.0,1.4,1.8,0.2,0.792982,0.853589,0.339569,0.7,45.2,5.4,4.4,4.952632,-0.081638,0.234601,4.9,282.3,9.8,7.2,8.394737,-0.103604,0.433186,8.4,478.5,1.513275,0.000000,0.363

In [72]:
fd.columns=[x.replace('"','').replace("(","").replace(")","").replace(",","").replace(" ","").replace(".","")  for x in fd.columns]

In [92]:
fd = fd.drop(del_cols,axis=1)

In [93]:
y_data = fd["behavior_id"]
x_data = fd.drop(["fragment_id", "behavior_id"], axis=1)

In [94]:
X_train,X_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3, random_state=10)

In [95]:
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

params = {
    'learning_rate': 0.1,
    'metric': 'multi_error',
    'objective': 'multiclass',
    'num_class': 19,
    'seed': 2020,
    'max_depth': 15,
    'num_leaves': 150,
    'lambda_l2': 0.1,
}

clf = lgb.train(params=params, train_set=train_data, valid_sets=[validation_data], num_boost_round=10000, verbose_eval=50, early_stopping_rounds=100, feval=lgb_acc_combo)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.300731	valid_0's lgb_acc_combo: 0.747932
[100]	valid_0's multi_error: 0.293419	valid_0's lgb_acc_combo: 0.75333
[150]	valid_0's multi_error: 0.287477	valid_0's lgb_acc_combo: 0.758031
[200]	valid_0's multi_error: 0.287477	valid_0's lgb_acc_combo: 0.758466
[250]	valid_0's multi_error: 0.287934	valid_0's lgb_acc_combo: 0.757944
Early stopping, best iteration is:
[159]	valid_0's multi_error: 0.286106	valid_0's lgb_acc_combo: 0.759598


In [96]:
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

params = {
    'learning_rate': 0.1,
    'metric': 'multi_error',
    'objective': 'multiclass',
    'num_class': 19,
    'seed': 2020,
    'max_depth': 20,
    'num_leaves': 300,
    'lambda_l2': 0.1,
}

clf = lgb.train(params=params, train_set=train_data, valid_sets=[validation_data], num_boost_round=10000, verbose_eval=50, early_stopping_rounds=100, feval=lgb_acc_combo)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.306216	valid_0's lgb_acc_combo: 0.742796
[100]	valid_0's multi_error: 0.292962	valid_0's lgb_acc_combo: 0.753613
[150]	valid_0's multi_error: 0.287934	valid_0's lgb_acc_combo: 0.758096
[200]	valid_0's multi_error: 0.287477	valid_0's lgb_acc_combo: 0.757574
Early stopping, best iteration is:
[115]	valid_0's multi_error: 0.28702	valid_0's lgb_acc_combo: 0.75864


In [97]:
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

params = {
    'learning_rate': 0.1,
    'metric': 'multi_error',
    'objective': 'multiclass',
    'num_class': 19,
    'seed': 2020,
    'max_depth': 15,
    'num_leaves': 300,
    'lambda_l2': 0.1,
}

clf = lgb.train(params=params, train_set=train_data, valid_sets=[validation_data], num_boost_round=10000, verbose_eval=50, early_stopping_rounds=100, feval=lgb_acc_combo)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.300731	valid_0's lgb_acc_combo: 0.747932
[100]	valid_0's multi_error: 0.293419	valid_0's lgb_acc_combo: 0.75333
[150]	valid_0's multi_error: 0.287477	valid_0's lgb_acc_combo: 0.758031
[200]	valid_0's multi_error: 0.287477	valid_0's lgb_acc_combo: 0.758466
[250]	valid_0's multi_error: 0.287934	valid_0's lgb_acc_combo: 0.757944
Early stopping, best iteration is:
[159]	valid_0's multi_error: 0.286106	valid_0's lgb_acc_combo: 0.759598


In [76]:
from sklearn.metrics import roc_auc_score, accuracy_score
y_pred = clf.predict(X_test, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_pred]
# print(y_pred)
print(accuracy_score(y_test, y_pred))

sum1 = 0
for y_t, y_p in zip(y_test, y_pred):
    sum1 += acc_combo(y_t, y_p)
    
combo_score = sum1/len(y_pred)
print(combo_score)

0.7189213893967094
0.7630147122834461


In [82]:
a = [x for x in zip(clf.feature_name(),clf.feature_importance())]

In [86]:
a.sort(key=lambda x: x[1],reverse=True)

In [91]:
del_cols = [x[0] for x in filter(lambda x: x[1]<=10, a)]

**添加特征列**

In [9]:
data_feat.shape

(7292, 2073)

In [9]:
data_feat = data.loc[:,["fragment_id","behavior_id"]].drop_duplicates()
feat_cols = [x for x in list(data.columns) if x not in ["fragment_id","behavior_id","time_point"]]
for feat_col in feat_cols:
    for agg_cal in ["max", "min","mean","skew","std","median","sum"]:
        feat_name, dic = get_dict(data, feat_col, agg_cal)
        data_feat[feat_name] = data_feat["fragment_id"].map(dic).values
#     data_feat[feat_col+"_gap"] = data_feat[feat_col+"_max"] - data_feat[feat_col+"_min"]

In [10]:
data_feat.head()

,fragment_id,behavior_id,acc_x_max,acc_x_min,acc_x_mean,acc_x_skew,acc_x_std,acc_x_median,acc_x_sum,acc_y_max,acc_y_min,acc_y_mean,acc_y_skew,acc_y_std,acc_y_median,acc_y_sum,acc_z_max,acc_z_min,acc_z_mean,acc_z_skew,acc_z_std,acc_z_median,acc_z_sum,acc_xg_max,acc_xg_min,acc_xg_mean,acc_xg_skew,acc_xg_std,acc_xg_median,acc_xg_sum,acc_yg_max,acc_yg_min,acc_yg_mean,acc_yg_skew,acc_yg_std,acc_yg_median,acc_yg_sum,acc_zg_max,acc_zg_min,acc_zg_mean,acc_zg_skew,acc_zg_std,acc_zg_median,acc_zg_sum,acc_max,acc_min,acc_mean,acc_skew,acc_std,acc_median,acc_sum,accg_max,accg_min,accg_mean,accg_skew,accg_std,accg_median,accg_sum,acc_x_y_max,acc_x_y_min,acc_x_y_mean,acc_x_y_skew,acc_x_y_std,acc_x_y_median,acc_x_y_sum,acc_x_z_max,acc_x_z_min,acc_x_z_mean,acc_x_z_skew,acc_x_z_std,acc_x_z_median,acc_x_z_sum,acc_y_z_max,acc_y_z_min,acc_y_z_mean,acc_y_z_skew,acc_y_z_std,acc_y_z_median,acc_y_z_sum,acc_xg_yg_max,acc_xg_yg_min,acc_xg_yg_mean,acc_xg_yg_skew,acc_xg_yg_std,acc_xg_yg_median,acc_xg_yg_sum,acc_xg_zg_max,acc_xg_zg_min,acc_xg_zg_mean,acc_xg_zg_skew,acc_xg_zg_std,acc_xg_zg_median,acc_xg_zg_sum,acc_yg_zg_max,acc_yg_zg_min,acc_yg_zg_mean,acc_yg_zg_skew,acc_yg_zg_std,acc_yg_zg_median,acc_yg_zg_sum
0,0,0,0.4,-0.9,-0.001754,-1.523203,0.194102,0.0,-0.1,0.2,-0.3,-0.001754,-0.497876,0.085547,-0.0,-0.1,0.7,-0.8,0.003509,-0.283199,0.294573,-0.0,0.2,1.3,-0.2,0.677193,-0.909003,0.200016,0.7,38.6,5.2,4.4,4.773684,0.284234,0.158707,4.8,272.1,9.3,7.2,8.508772,-0.746650,0.338738,8.5,485.0,0.953939,0.000000,0.281656,1.141795,0.225918,0.223607,16.054409,10.560776,8.635392,9.784581,-0.677138,0.293469,9.786726,557.721098,0.6,-0.8,1.947760e-18,-0.180893,0.226779,0.0,1.110223e-16,0.7,-0.8,-0.005263,0.071336,0.334034,0.0,-0.3,0.9,-0.9,-0.005263,0.082254,0.297862,0.0,-0.3,-3.6,-5.2,-4.096491,-1.104853,0.284708,-4.10,-233.5,-6.4,-9.5,-7.831579,-0.367830,0.436773,-7.8,-446.4,-2.5,-4.5,-3.735088,0.525371,0.388445,-3.8,-212.9
57,1,0,0.2,-0.5,-0.025000,-1.350853,0.109959,-0.0,-1.4,0.2,-0.1,0.026786,0.434311,0.058748,0.0,1.5,0.6,-0.5,0.078571,0.129004,0.184602,0.1,4.4,1.0,0.4,0.678571,-0.199871,0.126080,0.7,38.0,5.1,4.6,4.830357,0.411559,0.123465,4.8,270.5,9.4,8.1,8.519643,0.917398,0.241525,8.5,477.1,0.714143,0.000000,0.186749,1.626938,0.147525,0.141421,10.457923,10.521407,9.441398,9.819616,1.053017,0.202188,9.791067,549.898472,0.3,-0.6,-5.178571e-02,-0.894307,0.147699,0.0,-2.900000e+00,0.2,-0.6,-0.103571,-0.796679,0.186805,-0.1,-5.8,0.6,-0.5,-0.051786,0.272607,0.188767,0.0,-2.9,-3.8,-4.6,-4.151786,-0.268876,0.165135,-4.15,-232.5,-7.2,-8.9,-7.841071,-0.840256,0.283983,-7.8,-439.1,-3.0,-4.7,-3.689286,-0.431380,0.299502,-3.7,-206.6
113,2,0,0.7,-0.5,-0.028070,0.489693,0.239609,-0.0,-1.6,0.2,-0.6,-0.005263,-1.635879,0.136828,0.0,-0.3,1.2,-1.3,0.024561,-0.770588,0.396989,0.0,1.4,1.8,0.2,0.792982,0.853589,0.339569,0.7,45.2,5.4,4.4,4.952632,-0.081638,0.234601,4.9,282.3,9.8,7.2,8.394737,-0.103604,0.433186,8.4,478.5,1.513275,0.000000,0.363894,1.890966,0.316852,0.244949,20.741932,10.813418,8.823831,9.790638,-0.205059,0.355424,9.827004,558.066365,1.3,-0.6,-2.280702e-02,1.438085,0.313402,-0.1,-1.300000e+00,1.9,-1.6,-0.052632,0.469882,0.516204,-0.1,-3.0,1.5,-1.5,-0.029825,-0.005008,0.415746,0.0,-1.7,-2.9,-4.8,-4.159649,1.058967,0.413115,-4.20,-237.1,-5.9,-9.0,-7.601754,0.770910,0.605183,-7.8,-433.3,-2.0,-5.3,-3.442105,-0.109303,0.538831,-3.5,-196.2
170,3,0,0.5,-0.4,0.001818,0.072280,0.178980,0.0,0.1,0.4,-0.3,0.023636,1.126930,0.121661,-0.0,1.3,1.7,-0.8,0.094545,1.413450,0.496438,0.0,5.2,1.2,-0.6,0.410909,-0.555181,0.319532,0.4,22.6,5.5,4.5,5.049091,-0.054651,0.196141,5.0,277.7,9.5,7.5,8.340000,0.472631,0.414371,8.3,458.7,1.791647,0.000000,0.400589,1.973559,0.373524,0.282843,22.032386,10.585367,8.828363,9.767121,0.193839,0.360655,9.780082,537.191665,0.7,-0.8,-2.181818e-02,-0.394423,0.270590,0.0,-1.200000e+00,0.7,-2.1,-0.092727,-1.646655,0.566946,0.1,-5.1,1.1,-1.6,-0.070909,-1.041841,0.466537,0.0,-3.9,-3.3,-5.7,-4.638182,0.225728,0.465272,-4.70,-255.1,-6.3,-9.7,-7.929091,-0.492991,0.

In [11]:
# y_data = data_feat["behavior_id"]
x_data = data_feat.drop(["fragment_id","behavior_id"], axis=1)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import lightgbm as lgb

In [13]:
X_train,X_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3, random_state=10)

In [14]:
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

params = {
    'learning_rate': 0.05,
    'metric': 'multi_error',
    'objective': 'multiclass',
    'num_class': 19,
    'seed': 2020,
    'max_depth': 10,
    'num_leaves': 100,
    'lambda_l2': 0.1,
}

clf = lgb.train(params=params, train_set=train_data, valid_sets=[validation_data], num_boost_round=10000, verbose_eval=50, early_stopping_rounds=100, feval=lgb_acc_combo)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.303016	valid_0's lgb_acc_combo: 0.7421
[100]	valid_0's multi_error: 0.287934	valid_0's lgb_acc_combo: 0.755985
[150]	valid_0's multi_error: 0.277422	valid_0's lgb_acc_combo: 0.764451
[200]	valid_0's multi_error: 0.271938	valid_0's lgb_acc_combo: 0.770327
[250]	valid_0's multi_error: 0.264625	valid_0's lgb_acc_combo: 0.776813
[300]	valid_0's multi_error: 0.263254	valid_0's lgb_acc_combo: 0.777052
[350]	valid_0's multi_error: 0.260055	valid_0's lgb_acc_combo: 0.77999
[400]	valid_0's multi_error: 0.256399	valid_0's lgb_acc_combo: 0.782689
[450]	valid_0's multi_error: 0.250914	valid_0's lgb_acc_combo: 0.787151
[500]	valid_0's multi_error: 0.252742	valid_0's lgb_acc_combo: 0.785932
Early stopping, best iteration is:
[448]	valid_0's multi_error: 0.250914	valid_0's lgb_acc_combo: 0.787325


In [16]:
import gc

In [22]:
scores = []
imp = pd.DataFrame()
imp['feat'] = list(x_data.columns)
oof_train = np.zeros((len(X_train), 19))
preds = np.zeros((len(X_test), 19))
folds = 5
seeds = [2020,55] 

params = {
    'learning_rate': 0.05,
    'metric': 'multi_error',
    'objective': 'multiclass',
    'num_class': 19,
    'max_depth': 10,
    'num_leaves': 100,
    'lambda_l2': 0.1,
}

for seed in seeds:
    kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    for fold, (trn_idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
        x_trn, y_trn, x_val, y_val = X_train.iloc[trn_idx], y_train.iloc[trn_idx], X_train.iloc[val_idx], y_train.iloc[val_idx]
        train_set = lgb.Dataset(x_trn, y_trn)
        val_set = lgb.Dataset(x_val, y_val)
        
        model = lgb.train(params, train_set, num_boost_round=500000,
                          valid_sets=(train_set, val_set), early_stopping_rounds=250,
                          verbose_eval=50,feval=lgb_acc_combo)
        oof_train[val_idx] += model.predict(x_val) / len(seeds)
        preds += model.predict(X_test) / folds / len(seeds)
        scores.append(model.best_score['valid_1']['lgb_acc_combo'])
        imp['gain' + str(fold + 1)] = model.feature_importance(importance_type='gain')
        imp['split' + str(fold + 1)] = model.feature_importance(importance_type='split')
        del x_trn, y_trn, x_val, y_val, model, train_set, val_set
        gc.collect()

Training until validation scores don't improve for 250 rounds
[50]	training's multi_error: 0.0137154	training's lgb_acc_combo: 0.988512	valid_1's multi_error: 0.318315	valid_1's lgb_acc_combo: 0.729024
[100]	training's multi_error: 0.000489836	training's lgb_acc_combo: 0.999592	valid_1's multi_error: 0.299706	valid_1's lgb_acc_combo: 0.745394
[150]	training's multi_error: 0	training's lgb_acc_combo: 1	valid_1's multi_error: 0.285994	valid_1's lgb_acc_combo: 0.758034
[200]	training's multi_error: 0	training's lgb_acc_combo: 1	valid_1's multi_error: 0.279138	valid_1's lgb_acc_combo: 0.76433
[250]	training's multi_error: 0	training's lgb_acc_combo: 1	valid_1's multi_error: 0.264447	valid_1's lgb_acc_combo: 0.776923
[300]	training's multi_error: 0	training's lgb_acc_combo: 1	valid_1's multi_error: 0.265426	valid_1's lgb_acc_combo: 0.77557
[350]	training's multi_error: 0	training's lgb_acc_combo: 1	valid_1's multi_error: 0.261508	valid_1's lgb_acc_combo: 0.778835
[400]	training's multi_erro

In [23]:
imp['gain']  = imp[[f for f in imp.columns if 'gain' in f]].sum(axis=1)/folds
imp['split'] = imp[[f for f in imp.columns if 'split' in f]].sum(axis=1)
imp          = imp.sort_values(by=['gain'], ascending=False)
imp[['feat','gain', 'split']]

,feat,gain,split
47,acc_median,13818.974359,24251
28,acc_yg_max,12899.405770,21399
55,accg_sum,10217.409045,67698
29,acc_yg_min,9474.205439,27781
11,acc_y_std,9414.935584,31260
...,...,...,...
68,acc_x_z_median,325.318499,8388
5,acc_x_median,325.125222,5924
75,acc_y_z_median,293.643794,8780
61,acc_x_y_median,272.402179,6199


In [24]:
preds = [list(x).index(max(x)) for x in preds]
print(accuracy_score(y_test, preds))
sum0 = 0
for y_t, y_p in zip(y_test, preds):
    sum0 += acc_combo(y_t, y_p)
combo_score0 = sum0/len(y_pred)
print(combo_score0)

0.7372029250457038
0.7769870288151783


In [25]:
from sklearn.metrics import roc_auc_score, accuracy_score
y_pred = clf.predict(X_test, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_pred]
# print(y_pred)
print(accuracy_score(y_test, y_pred))

0.7490859232175503


In [26]:
sum1 = 0
for y_t, y_p in zip(y_test, y_pred):
    sum1 += acc_combo(y_t, y_p)

In [27]:
combo_score = sum1/len(y_pred)
print(combo_score)

0.7873248019500264


## 输出判断错误的样例

In [28]:
len(y_test)

2188

In [29]:
ct = 0
for index, (y_t, y_p) in zip(y_test.index, zip(y_test, y_pred)):
    if y_t == y_p:
        ct+=1
    else:
        print(index, y_t, y_p)

10253 0 6
229720 9 6
79304 3 2
28906 1 6
133042 5 7
168181 6 12
353718 15 8
317217 12 4
147320 6 12
335152 14 12
398862 17 8
130574 5 1
4885 0 4
19936 1 6
174252 6 1
340374 14 4
248233 10 6
146363 6 4
320642 13 12
205332 8 7
415297 18 4
398682 17 16
217244 8 16
154244 6 12
194889 7 6
332223 13 12
416880 18 13
186648 7 9
351894 15 4
15121 1 0
30476 1 6
380571 16 4
85071 3 0
359480 15 0
40485 1 7
163328 6 12
356019 15 12
46163 1 4
203064 8 2
331929 13 12
188364 7 17
49012 1 13
336503 14 12
227472 9 7
185631 7 12
411495 18 12
225743 9 6
48837 1 3
252143 11 15
227117 9 7
10134 0 15
45353 1 6
207347 8 17
178539 6 9
403769 17 12
396408 17 13
6744 0 1
399397 17 12
380511 16 13
79363 3 2
356370 15 4
159614 6 12
9904 0 12
79655 3 2
234937 9 6
205734 8 4
135900 5 6
344369 15 1
85251 3 6
153441 6 0
353838 15 2
8639 0 3
156833 6 0
66043 2 17
210079 8 7
232851 9 6
121581 5 8
57275 2 5
41145 1 6
290308 12 17
394062 17 13
390339 17 12
411728 18 1
172378 6 1
203540 8 17
7264 0 6
82196 3 9
235807 10 6


In [30]:
ct

1639

**输出特征重要度**

In [33]:
from jhplt import plt_feat_import
bar = plt_feat_import(clf, max_cols=99, max_print=10)
bar.load_javascript()

[('accg_sum', 14412), ('accg_mean', 10491), ('accg_median', 9071), ('acc_skew', 7971), ('acc_yg_skew', 6855), ('acc_zg_sum', 6852), ('acc_y_std', 6798), ('acc_y_skew', 6763), ('acc_xg_skew', 6259), ('acc_xg_yg_std', 6140)]


C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [34]:
bar.render_notebook()

**分数记录**  
+ 0711baseline
  - acc: 0.6814442413162706
  - combo: 0.7299556019848483

## 结果预估 

In [177]:
test_data = pd.read_csv(data_path+"sensor_test\\sensor_test.csv")
test_data["acc"] = (test_data['acc_x'] ** 2 + test_data['acc_y'] ** 2 + test_data['acc_z'] ** 2) ** 0.5
test_data['accg'] = (test_data['acc_xg'] ** 2 + test_data['acc_yg'] ** 2 + test_data['acc_zg'] ** 2) ** 0.5

test_data_feat = test_data.loc[:,["fragment_id"]].drop_duplicates()
feat_cols = [x for x in list(test_data.columns) if x not in ["fragment_id","time_point"]]
for feat_col in feat_cols:
    for agg_cal in ["max", "min","mean","skew","std","median","sum"]:
        feat_name, dic = get_dict(test_data, feat_col, agg_cal)
        test_data_feat[feat_name] = test_data_feat["fragment_id"].map(dic).values
#     data_feat[feat_col+"_gap"] = data_feat[feat_col+"_max"] - data_feat[feat_col+"_min"]

In [178]:
test_data_feat.head()

,fragment_id,acc_x_max,acc_x_min,acc_x_mean,acc_x_skew,acc_x_std,acc_x_median,acc_x_sum,acc_y_max,acc_y_min,acc_y_mean,acc_y_skew,acc_y_std,acc_y_median,acc_y_sum,acc_z_max,acc_z_min,acc_z_mean,acc_z_skew,acc_z_std,acc_z_median,acc_z_sum,acc_xg_max,acc_xg_min,acc_xg_mean,acc_xg_skew,acc_xg_std,acc_xg_median,acc_xg_sum,acc_yg_max,acc_yg_min,acc_yg_mean,acc_yg_skew,acc_yg_std,acc_yg_median,acc_yg_sum,acc_zg_max,acc_zg_min,acc_zg_mean,acc_zg_skew,acc_zg_std,acc_zg_median,acc_zg_sum,acc_max,acc_min,acc_mean,acc_skew,acc_std,acc_median,acc_sum,accg_max,accg_min,accg_mean,accg_skew,accg_std,accg_median,accg_sum
0,0,2.4,-3.0,0.020690,-0.352313,1.008170,0.05,1.2,1.2,-1.9,-0.163793,-0.264509,0.812123,0.0,-9.5,4.3,-3.5,0.103448,0.310273,1.728093,0.0,6.0,4.1,-3.3,0.631034,0.076648,1.380720,0.40,36.6,6.0,2.1,3.881034,0.036160,0.910650,3.85,225.1,14.4,3.4,8.879310,0.109771,2.106694,8.75,515.0,4.505552,0.424264,1.894651,0.506177,1.023814,1.746425,109.889781,14.763807,6.014981,9.913074,0.175057,1.771438,10.071741,574.958306
58,1,0.2,-0.2,-0.003636,-0.339436,0.094209,0.00,-0.2,0.2,-0.3,-0.020000,-1.622548,0.080277,0.0,-1.1,0.4,-0.3,-0.001818,0.183871,0.155743,0.0,-0.1,-0.7,-1.2,-0.903636,-0.104787,0.108804,-0.90,-49.7,4.7,4.4,4.512727,0.287364,0.086184,4.50,248.2,8.7,8.0,8.356364,-0.272781,0.147527,8.40,459.6,0.412311,0.000000,0.169616,0.186859,0.103429,0.200000,9.328892,9.804081,9.233093,9.541062,-0.176265,0.137278,9.565041,524.758401
113,2,1.3,-0.5,0.126786,0.981016,0.268661,0.10,7.1,0.4,-0.3,0.016071,0.116605,0.139840,0.0,0.9,2.0,-1.0,-0.010714,1.590047,0.419818,0.0,-0.6,2.2,-0.7,1.385714,-2.726484,0.406953,1.45,77.6,3.9,1.5,3.162500,-2.609751,0.399801,3.20,177.1,11.7,7.8,9.203571,1.876808,0.491737,9.20,515.4,2.083267,0.000000,0.379726,2.529841,0.371367,0.263896,21.264674,11.816514,8.830629,9.850261,1.681836,0.399589,9.803313,551.614624
169,3,0.3,-0.3,-0.028070,-0.056283,0.111410,0.00,-1.6,0.2,-0.3,-0.038596,-1.112233,0.090147,0.0,-2.2,0.5,-0.5,0.028070,-0.503849,0.177033,0.0,1.6,0.3,0.0,0.136842,-0.130838,0.099340,0.20,7.8,6.1,5.6,5.842105,-0.070677,0.105131,5.90,333.0,7.9,7.1,7.480702,-0.243198,0.173657,7.50,426.4,0.519615,0.000000,0.188687,0.519045,0.136972,0.200000,10.755163,9.841240,9.168424,9.494134,-0.163860,0.147636,9.502105,541.165655
226,4,2.7,-1.8,-0.050000,0.720033,0.821429,0.00,-3.0,1.0,-1.7,-0.386667,-0.213674,0.603792,-0.3,-23.2,2.4,-1.4,0.313333,0.277146,0.864844,0.3,18.8,6.8,0.9,3.068333,0.708651,1.075536,3.10,184.1,6.3,3.6,4.958333,0.069370,0.563672,4.90,297.5,10.0,5.9,7.886667,0.041329,0.939792,7.90,473.2,2.875761,0.223607,1.276421,0.433955,0.620347,1.185515,76.585242,12.185647,8.322259,9.891203,0.452428,0.832659,9.836409,593.472191


In [179]:
test_data_feat.shape

(7500, 57)

In [180]:
test_fragment_id = test_data_feat["fragment_id"].values

In [181]:
y_pred = clf.predict(test_data_feat.drop("fragment_id", axis=1).values)

In [182]:
y_pred = [list(x).index(max(x)) for x in y_pred]

In [183]:
res = [(x, y) for x,y in zip(test_fragment_id,y_pred)]

In [184]:
pd.DataFrame(res, columns=["fragment_id", "behavior_id"]).to_csv(data_path+"result\\res_baseline_0711_{}.csv".format(combo_score), index=False)